In [ ]:
import os
from glob import glob
import sklearn as sk
import nibabel as nib
import nilearn as nil
import numpy as np
import pandas as pd
import scipy
from time import time
import json
from itertools import product
import datetime

from scripts.ComBat_experiments import *

from joblib import Parallel, delayed

import re, string
from scripts.evaluation_classifier import Evaluater
from scripts.pycombat import Combat

import matplotlib.pyplot as plt 
from matplotlib.ticker import PercentFormatter

from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, LeaveOneGroupOut
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_predict, RepeatedStratifiedKFold

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, KernelCenterer, LabelEncoder, label_binarize
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, jaccard_score
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_fscore_support, classification_report
from nilearn import plotting

from nilearn.maskers import NiftiLabelsMasker, NiftiMasker

import sklearn

In [3]:
sklearn.__version__ # '1.0.2'

'1.0.2'

In [6]:
def filter_filelist(filelist, feature_label, file_filter=[]):
    filter_list = np.append(feature_label.lower(), file_filter)
    if 'seedCorr' in feature_label:
        mask = [np.all([f in file.split('task-')[-1].lower() for f in filter_list]) for file in rename_seed_files(filelist)]
    else:
        mask = [np.all([f in file.split('task-')[-1].lower() for f in filter_list]) for file in filelist]
    return sorted(np.array(filelist)[mask])



def repeated_group_stratified_KFold_splits(y, groups, n_folds=5, n_repeats=20, seed=0, verbose=True):
    
    # Combine class and site labels
    labels_to_strat = np.array([site + "_and_" + str(int(label)) for site, label in zip(groups, y)])
    
    # Check if there are any site * label combinations with less than 2 counts
    vals, counts = np.unique(labels_to_strat, return_counts=True)
    vals_to_filter = vals[counts < 2]
    
    # Stratify these only by site instead
    if len(vals_to_filter) > 0:
        if verbose:
            print("Warning! Detected site * label combinations with less than 2 counts:")
            print(f"{vals[counts < 2]} : {counts[counts < 2]}")
        for val in vals_to_filter:
            new_val = val.split('_and_')[0] + '_and_' + str(int(not bool(int(val.split('_and_')[1]))))
            labels_to_strat[labels_to_strat == val] = new_val
            if verbose:
                print(f"Replaced {val} with {new_val}")
        
    # Create and return repeated group stratified KFold splits
    cv_splits = []
    ssKFold_cv = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=n_repeats, random_state=seed)
    for i, (train, test) in enumerate(ssKFold_cv.split(y=labels_to_strat, X=np.zeros_like(labels_to_strat))):
        cv_splits.append((train, test))
    return(cv_splits)



def group_stratified_shuffle_cv_splits(y, groups, n_splits=100, train_size=0.8, test_size=0.2, seed=0, 
                                       verbose=True):
    
    # Combine class and site labels
    labels_to_strat = np.array([site + "_and_" + str(int(label)) for site, label in zip(groups, y)])
    
    # Check if there are any site * label combinations with less than 2 counts
    vals, counts = np.unique(labels_to_strat, return_counts=True)
    vals_to_filter = vals[counts < 2]
    
    # Stratify these only by site instead
    if len(vals_to_filter) > 0:
        if verbose:
            print("Warning! Detected site * label combinations with less than 2 counts:")
            print(f"{vals[counts < 2]} : {counts[counts < 2]}")
        for val in vals_to_filter:
            new_val = val.split('_and_')[0] + '_and_' + str(int(not bool(int(val.split('_and_')[1]))))
            labels_to_strat[labels_to_strat == val] = new_val
            if verbose:
                print(f"Replaced {val} with {new_val}")
        
    # Create and return stratified splits
    cv_splits = []
    ss_cv = StratifiedShuffleSplit(n_splits=n_splits, train_size=train_size, test_size=test_size, 
                                   random_state=seed)
    for i, (train, test) in enumerate(ss_cv.split(y=labels_to_strat, X=np.zeros_like(labels_to_strat))):
        cv_splits.append((train, test))
    return(cv_splits)

In [7]:
def merge_bool_masks(*masks):
    return np.array([all(tup) for tup in zip(*masks)])


def has_N_samples_per_class(covariates_df, class_label, subject_mask=None, N_threshold=10, verbose=True):
    
    groups = np.array(covariates_df.site_id.values)
    
    if subject_mask is not None:
        tmp_df = covariates_df.loc[subject_mask, ['ID', class_label, 'Age', 'Sex', 'site_id']]
    else:
        tmp_df = covariates_df.loc[:, ['ID', class_label, 'Age', 'Sex', 'site_id']]
    
    all_sites = np.unique(tmp_df.site_id)
    unique_class_values = sorted(np.unique(tmp_df[class_label]))
    assert len(unique_class_values) == 2

    # Store number of included subjects per class and site in Dataframe
    counts_df = tmp_df.groupby(['site_id', class_label]).size()
    counts_df = pd.DataFrame(counts_df)
    counts_df = counts_df.reset_index()
    counts_df = counts_df.pivot(index='site_id', columns=class_label)
    counts_df.columns = counts_df.columns.droplevel(0)
    counts_df.columns.name = None
    counts_df = counts_df.reset_index()

    included_sites = counts_df.loc[(counts_df[unique_class_values[0]] >= N_threshold) & 
                                   (counts_df[unique_class_values[1]] >= N_threshold)]['site_id'].values
    excluded_sites = list(set(all_sites).difference(included_sites))
    
    counts_df = counts_df.loc[counts_df.site_id.isin(included_sites)]

    site_mask = [g not in excluded_sites for g in groups]
    
    if verbose:
        N_excluded = len(tmp_df.loc[tmp_df.site_id.isin(excluded_sites)])
        print("Excluded {} subjects belonging to {} different sites:\n\n{}".format(N_excluded, 
                                                                                   len(excluded_sites), 
                                                                                   '\n'.join(excluded_sites)))
    return site_mask, counts_df
    

def ensure_folder(folder_dir):
    if not os.path.exists(folder_dir):
        os.makedirs(folder_dir)

In [8]:
MAIN_OUTPUT_DIR = '/data/wbbruin/Desktop/ENIGMA_HALFPIPE_OUTPUTS/'
ENIGMA_ATLASES_DIR = '/data/wbbruin/Desktop/ME_RSFMRI_TEDANA/atlases_and_seeds/'

FEATURE_LABELS = ['corrMatrix', 'fALFF', 'seedCorr', 'dualReg', 'reHo']

# Specify directory to store group summary of halfpipe derivties (csv) and parsed features into  NumPy .npy format
SAVE_DIR = '/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/Data/'

# Regex to remove all non-alphanumeric characters
pattern = re.compile('[\W_]+')

# Extract the 55 seed labels, reformat these to lowercase and remove all non-alphanumeric characters
SEED_PATHS = sorted(glob(os.path.join(ENIGMA_ATLASES_DIR, '*seed_2009*')))
assert len(SEED_PATHS) == 55 
SEED_LABELS = [pattern.sub('', os.path.basename(seed).split('_seed_2009')[0].lower()) 
               for seed in SEED_PATHS]
assert len(SEED_LABELS) == len(np.unique(SEED_LABELS))
sorted_idx = np.argsort(SEED_LABELS)
SEED_PATHS = np.array(SEED_PATHS)[sorted_idx]
SEED_LABELS = np.array(SEED_LABELS)[sorted_idx]

# Extract the 3 atlas labels for connectomes
CONNECTOME_PATHS = sorted(glob(os.path.join(ENIGMA_ATLASES_DIR, 'tpl-MNI152NLin2009cAsym_atlas-*.nii.gz')))
assert len(CONNECTOME_PATHS) == 3
CONNECTOME_LABELS = [pattern.sub('', os.path.basename(atlas).split('atlas-')[-1].split('.nii.gz')[0].lower()) 
                     for atlas in CONNECTOME_PATHS]
CONNECTOME_LABELS = [c.split('dseg')[0] for c in CONNECTOME_LABELS]

sorted_idx = np.argsort(CONNECTOME_LABELS)
CONNECTOME_PATHS = np.array(CONNECTOME_PATHS)[sorted_idx]
CONNECTOME_LABELS = np.array(CONNECTOME_LABELS)[sorted_idx]


# Extract the 14 dual regression component labels
DUAL_REGRESSION_IC_LABELS = [str.lower('FINDIca_component-'+str(i).zfill(2)) for i in np.arange(14) + 1]

In [9]:
# !!! Load most recent pooled data

pooled_data_main_dir = '/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/Pooled_Data'
pooled_data_dirs = os.listdir('/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/Pooled_Data')

if len(pooled_data_dirs) > 0:
    
    print("Loading most recent pooled data...")
    most_recent_idx = np.argmax([datetime.datetime.strptime(d, "%d_%m_%Y") for d in pooled_data_dirs])
    print("Found data stored @ {}".format(pooled_data_dirs[most_recent_idx]))
    pooled_data_dir = os.path.join(pooled_data_main_dir, pooled_data_dirs[most_recent_idx])
    pooled_data_path = os.path.join(pooled_data_dir, 'pooled_data.npz')
    
    data = np.load(pooled_data_path, allow_pickle=True)

    included_subject_ids=data['included_subject_ids']
    
    roi_labels=data['roi_labels']
    network_labels=data['network_labels']
    
    network_ordering_dict=data['network_ordering_dict'].item()

    fALFF_per_ROI=data['fALFF_per_ROI']
    fALFF_per_network=data['fALFF_per_network']

    reHo_per_ROI=data['reHo_per_ROI']
    reHo_per_network=data['reHo_per_network']

    CC_matrix=data['CC_matrix']

    within_network_correlations=data['within_network_correlations']
    within_network_correlations_labels=data['within_network_correlations_labels']

    between_network_correlations=data['between_network_correlations']
    between_network_correlations_labels=data['between_network_correlations_labels']

    filtered_atlas_img = nib.load(os.path.join(pooled_data_dir, 'included_regions_atlas.nii.gz'))
    lut_df = pd.read_csv(os.path.join(pooled_data_dir, 'Schaefer2018_400Parcels_17Networks_LUT.csv'))

    included_subjects_derivates_df = pd.read_csv(os.path.join(pooled_data_dir, 
                                                              'included_subjects_derivates.csv'))
    included_subjects_covariates_df = pd.read_csv(os.path.join(pooled_data_dir, 
                                                               'included_subjects_covariates.csv'))

    N_subjects = included_subjects_derivates_df.shape[0]
    N_sites = len(included_subjects_derivates_df.site_id.unique())

    print("Loaded pooled data for {} subjects from {} sites...".format(N_subjects, N_sites))

    print("Done!")


Loading most recent pooled data...
Found data stored @ 01_03_2022
Loaded pooled data for 2148 subjects from 33 sites...
Done!


### Prepare features for analysis

In [10]:
n_ROIs = len(roi_labels)

assert n_ROIs == CC_matrix.shape[-1]

print(n_ROIs)

318


In [11]:
unique_network_labels = []

for label in network_ordering_dict.keys():
    if label not in network_labels:
        continue
    else:
        unique_network_labels.append(label)
        
N_networks = len(unique_network_labels)

assert N_networks == len(np.unique(network_labels))
assert N_networks == reHo_per_network.shape[-1]
assert N_networks == fALFF_per_network.shape[-1]

print(N_networks)

18


In [12]:
# Unravel pair-wise ROI-to-ROI FC as 1d features.

# For a matrix of size n x n, the number of elements in the lower triangle is
print(CC_matrix.shape[-1] * (CC_matrix.shape[-1] - 1) / 2)

# Create mask for lower triangular of corremation matrix. Skip diagonal (auto) correlations.
tril_mask = np.tril(np.ones(CC_matrix.shape[-2:]), k=-1).astype(bool)
print(np.sum(tril_mask))

# Make sure IDs are matched so we can extract diagnosis and other covariates easily
assert np.all(included_subjects_derivates_df.site_subject_ID.values == included_subject_ids)
assert np.all(included_subjects_covariates_df.site_subject_ID.values == included_subject_ids)
assert np.all(included_subjects_derivates_df.site_subject_ID.values == 
              included_subjects_covariates_df.site_subject_ID.values)

# Take lower triangular of FC matrix
pairwise_ROI_to_ROI_FC_vec = CC_matrix[..., tril_mask]

print(pairwise_ROI_to_ROI_FC_vec.shape)

50403.0
50403
(2148, 50403)


In [13]:
(318*317)/2

50403.0

In [14]:
assert np.all(included_subjects_derivates_df.site_id.values ==  included_subjects_covariates_df.site_id.values)

In [15]:
# We can choose to investigate within and between network FC separately, or combined

print(between_network_correlations.shape)

# Unravel pair-wise between-network FC correlations as 1d features. Skip diagonal!
tril_mask = np.tril(np.ones(between_network_correlations.shape[-2:]), k=-1).astype(bool)
pairwise_between_network_FC_vec = between_network_correlations[..., tril_mask]

print(pairwise_between_network_FC_vec.shape)

# Within-network FC correlations are already 1 features
print(within_network_correlations.shape)


# Combine between- and within network correlation vectors
pairwise_network_FC_vec = np.c_[pairwise_between_network_FC_vec, within_network_correlations]
print(pairwise_network_FC_vec.shape)

(2148, 18, 18)
(2148, 153)
(2148, 16)
(2148, 169)


In [16]:
# Confirm that all subjects included so far have diagnosis status

print(np.unique(included_subjects_covariates_df.Diagnosis))
assert sum(included_subjects_covariates_df.Diagnosis.isna()) == 0

[1. 2.]


In [17]:
# Extract Diagnosis status, and Sex & Age covariates

# "Diagnosis" recoded from (OCD=1, HC=2) to (OCD=1, HC=0)
Dx_vec = np.array(included_subjects_covariates_df.Diagnosis.replace(2, 0).values, dtype=int)

# "Sex" recoded from (1=male, 2=female) to (1=male, 0=female)
Sex_vec = np.array(included_subjects_covariates_df.Sex.replace(2, 0).values, dtype=int)

# "Age" scaled by / 100
Age_vec = np.array(included_subjects_covariates_df.Age) / 100.

# "Year of Education" scaled by / 30 (max = 27 years)
Edu_vec = np.array(included_subjects_covariates_df.Years_of_education) / 30.

# Site IDs
groups_vec = np.array(included_subjects_derivates_df.site_id.values)

# There are subjects missing either Sex, Age, Years of Education and/or all
IDs_missing_Sex = included_subjects_covariates_df.loc[
                                    (included_subjects_covariates_df.Sex.isna())].site_subject_ID.values

IDs_missing_Age = included_subjects_covariates_df.loc[
                                    (included_subjects_covariates_df.Age.isna())].site_subject_ID.values

IDs_missing_Edu = included_subjects_covariates_df.loc[
                                    (included_subjects_covariates_df.Years_of_education.isna(
                                    ))].site_subject_ID.values

IDs_missing_age_and_sex_covars = set(IDs_missing_Age).union(IDs_missing_Sex)
IDs_missing_all_covars = set(IDs_missing_Age).union(IDs_missing_Sex).union(IDs_missing_Edu)

print("{} subjects miss Sex covariate".format(len(IDs_missing_Sex)))
print("{} subjects miss Age covariate".format(len(IDs_missing_Age)))
print("{} subjects miss Years of Education covariate".format(len(IDs_missing_Edu)))
print("{} subjects miss ALL covariates".format(len(IDs_missing_all_covars)))

0 subjects miss Sex covariate
0 subjects miss Age covariate
178 subjects miss Years of Education covariate
178 subjects miss ALL covariates


In [18]:
IDs_missing_FD = included_subjects_derivates_df.loc[
                                    (included_subjects_derivates_df.fd_mean.isna(
                                    ))].site_subject_ID.values

FD_vec = np.array(included_subjects_derivates_df.fd_mean.values)
print(IDs_missing_FD)

[]


In [19]:
# Extract info for sensitivity analyses: medication status, age of onset, years of education, severity

# "Medication status at time of scan  (1=naive, 2=unmedicated, 3=medicated)

is_unmed_pt = ((included_subjects_covariates_df.Medication_status.isin([1, 2])) & 
               (included_subjects_covariates_df.Diagnosis == 1))

is_med_pt = ((included_subjects_covariates_df.Medication_status.isin([3])) & 
             (included_subjects_covariates_df.Diagnosis == 1))

is_unmed_hc = ((included_subjects_covariates_df.Medication_status.isin([1, 2])) & 
               (included_subjects_covariates_df.Diagnosis == 2))

is_med_hc = (included_subjects_covariates_df.Medication_status.isin([3]) & 
            (included_subjects_covariates_df.Diagnosis == 2))

# Specific for unmedicated OCD vs HC or medicated OCD vs HC
is_med_pt_or_unmed_hc_mask = is_med_pt | is_unmed_hc 
is_unmed_pt_or_unmed_hc_mask = is_unmed_pt | is_unmed_hc

medication_groups_masks_dict = {
                                 'unmed_pt': is_unmed_pt,
                                 'med_pt': is_med_pt,
                                 'unmed_hc': is_unmed_hc,
                                 'med_hc': is_med_hc
                                }

medication_groups = list(medication_groups_masks_dict.keys())

for medication_group in medication_groups:
    print("N {} samples: {}".format(medication_group, sum(medication_groups_masks_dict[medication_group])))
    

Dx_med_filter_dict = {
                      'Med_OCD_Unmed_HC' : is_med_pt_or_unmed_hc_mask,
                      'Unmed_OCD_Unmed_HC' : is_unmed_pt_or_unmed_hc_mask
                      }

Dx_med_filter_labels = list(Dx_med_filter_dict.keys())

for Dx_med_filter in Dx_med_filter_labels:
    print("N {} samples: {}".format(Dx_med_filter, sum(Dx_med_filter_dict[Dx_med_filter])))

N unmed_pt samples: 555
N med_pt samples: 528
N unmed_hc samples: 748
N med_hc samples: 0
N Med_OCD_Unmed_HC samples: 1276
N Unmed_OCD_Unmed_HC samples: 1303


In [20]:
# Define median severity for patients

median_sev = included_subjects_covariates_df.loc[(~included_subjects_covariates_df['Y-BOCS'].isna()) &
                                                 (included_subjects_covariates_df.Diagnosis == 1)]\
                                                ['Y-BOCS'].median()
print(median_sev)


# Previously for structural ENIGMA-OCD we defined severity splits with median 24:

# (YBOCS <= 24; mild-moderate) and high severity (YBOCS > 24; moderate-severe) OCD;
# y = np.array(y > 24, dtype=int)

# Now we use YBOCS <= 25 (mild-moderate) and > 25 (moderate-severe)
is_low_sev_pt = ((included_subjects_covariates_df['Y-BOCS'] <= median_sev) & 
                 (included_subjects_covariates_df.Diagnosis == 1))
is_high_sev_pt = ((included_subjects_covariates_df['Y-BOCS'] > median_sev) & 
                  (included_subjects_covariates_df.Diagnosis == 1))

sev_groups_masks_dict = {
                         'low_sev_pt': is_low_sev_pt,
                         'high_sev_pt': is_high_sev_pt
                        }

sev_groups = list(sev_groups_masks_dict.keys())

for sev_group in sev_groups:
    print("N {} samples: {}".format(sev_group, sum(sev_groups_masks_dict[sev_group])))
    
# Only want to look into high/low sevety patients vs unmedicated subjects!
is_low_sev_pt_or_unmed_hc_mask = is_low_sev_pt | is_unmed_hc 
is_high_sev_pt_or_unmed_hc_mask = is_high_sev_pt | is_unmed_hc
    
Dx_sev_filter_dict = {
                      'Low_Sev_OCD_Unmed_HC' : is_low_sev_pt_or_unmed_hc_mask,
                      'High_Sev_OCD_Unmed_HC' : is_high_sev_pt_or_unmed_hc_mask
                      }

Dx_sev_filter_labels = list(Dx_sev_filter_dict.keys())

for Dx_sev_filter in Dx_sev_filter_labels:
    print("N {} samples: {}".format(Dx_sev_filter, sum(Dx_sev_filter_dict[Dx_sev_filter])))

25.0
N low_sev_pt samples: 579
N high_sev_pt samples: 494
N Low_Sev_OCD_Unmed_HC samples: 1327
N High_Sev_OCD_Unmed_HC samples: 1242


In [21]:
# Define masks for early and late onset OCD:

# Early-onset OCD patients / children (onset before age 18) to controls
# Late-onset OCD patients / adult (onset at age 18 or older)

# Consistent with previous ENIGMA-OCD analyses, I used y = np.array(y >= 18, dtype=int) for structural

# Now we only have coding for 1 or 2 (1=Child-onset, 2=Adult-onset, N/A)

is_early_AO_pt = ((included_subjects_covariates_df.Age_of_onset == 1) & 
                  (included_subjects_covariates_df.Diagnosis == 1))
is_late_AO_pt = ((included_subjects_covariates_df.Age_of_onset == 2) & 
                 (included_subjects_covariates_df.Diagnosis == 1))

AO_groups_masks_dict = {
                        'early_AO_pt': is_early_AO_pt,
                        'late_AO_pt': is_late_AO_pt
                        }

AO_groups = list(AO_groups_masks_dict.keys())

for AO_group in AO_groups:
    print("N {} samples: {}".format(AO_group, sum(AO_groups_masks_dict[AO_group])))
    
    
# Only want to look into high/low sevety patients vs unmedicated subjects!
is_early_AO_pt_or_unmed_hc_mask = is_early_AO_pt | is_unmed_hc 
is_late_AO_pt_or_unmed_hc_mask = is_late_AO_pt | is_unmed_hc
    
Dx_AO_filter_dict = {
                      'Early_AO_OCD_Unmed_HC' : is_early_AO_pt_or_unmed_hc_mask,
                      'Late_AO_OCD_Unmed_HC' : is_late_AO_pt_or_unmed_hc_mask
                      }

Dx_AO_filter_labels = list(Dx_AO_filter_dict.keys())

for Dx_AO_filter in Dx_AO_filter_labels:
    print("N {} samples: {}".format(Dx_AO_filter, sum(Dx_AO_filter_dict[Dx_AO_filter])))

N early_AO_pt samples: 492
N late_AO_pt samples: 478
N Early_AO_OCD_Unmed_HC samples: 1240
N Late_AO_OCD_Unmed_HC samples: 1226


In [22]:
# Define masks for specific age groups
is_pediatric_mask = included_subjects_covariates_df.Age_Sample_WBB.isin([1]).values
is_adolescent_mask = included_subjects_covariates_df.Age_Sample_WBB.isin([2]).values
is_adult_mask = included_subjects_covariates_df.Age_Sample_WBB.isin([3]).values
is_pooled_mask = np.ones_like(is_adult_mask, dtype=bool) 
is_pediatric_or_adolescent_mask = is_pediatric_mask | is_adolescent_mask # We will use this for pediatric analyses

# Create dict so we can access masks easily
age_groups = ['pooled', 'adult', 'pediatric']
age_group_masks_dict = dict(zip(age_groups, [is_pooled_mask, is_adult_mask, is_pediatric_or_adolescent_mask]))

for age_group in age_groups:
    print("N {} samples: {}".format(age_group, sum(age_group_masks_dict[age_group])))

N pooled samples: 2148
N adult samples: 1897
N pediatric samples: 251


In [23]:
print(sum(is_pediatric_mask), sum(is_adolescent_mask), sum(is_adult_mask))

37 214 1897


In [24]:
print(fALFF_per_ROI.min(), fALFF_per_ROI.max())

-5.134157657623291 5.390778320527733


In [25]:
print(reHo_per_ROI.min(), reHo_per_ROI.max())

-3.3230038681054084 5.194765826580683


In [26]:
# Finally, in order to use fALFF and reHo features for classfication, we need to scale for better runtime:

X_min, X_max = -5, 5

fALFF_per_ROI_scaled = (fALFF_per_ROI - X_min) / (X_max - X_min)
fALFF_per_network_scaled = (fALFF_per_network - X_min) / (X_max - X_min)
reHo_per_ROI_scaled = (reHo_per_ROI - X_min) / (X_max - X_min)
reHo_per_network_scaled = (reHo_per_network - X_min) / (X_max - X_min)

In [27]:
# Add demographic feature set using only Age and Sex as a "baseline" comparison

demographic_data = np.c_[Age_vec, Sex_vec]
demographic_data.shape

(2148, 2)

In [50]:
# Create dict that maps feature labels with data

feature_sets_dict = {
#                      'demographical': demographic_data,
                     'ROI-to-ROI-FC': pairwise_ROI_to_ROI_FC_vec,
                     'network-FC': pairwise_network_FC_vec,
                     'reHo-ROI': reHo_per_ROI_scaled,
                     'reHo-network': reHo_per_network_scaled,
                     'fALFF-ROI': fALFF_per_ROI_scaled,
                     'fALFF-network': fALFF_per_network_scaled,
                     }


feature_labels = list(feature_sets_dict.keys())
for feature_label in feature_labels:
    print("{} features shape: {}".format(feature_label, feature_sets_dict[feature_label].shape))

ROI-to-ROI-FC features shape: (2148, 50403)
network-FC features shape: (2148, 169)
reHo-ROI features shape: (2148, 318)
reHo-network features shape: (2148, 18)
fALFF-ROI features shape: (2148, 318)
fALFF-network features shape: (2148, 18)


In [51]:
for feature_label in feature_labels:
    print("{} features min: {}, max: {}".format(feature_label, feature_sets_dict[feature_label].min(),
                                                feature_sets_dict[feature_label].max()))

ROI-to-ROI-FC features min: -1.7602020636513347, max: 2.5371584000790963
network-FC features min: -0.6851071322346465, max: 1.4840383752234771
reHo-ROI features min: 0.16769961318945917, max: 1.0194765826580683
reHo-network features min: 0.31048786640167236, max: 0.7182867317721732
fALFF-ROI features min: -0.013415765762329102, max: 1.0390778320527734
fALFF-network features min: 0.25634422302246096, max: 0.8620645244242416


In [30]:
CLF_RESULTS_DIR = os.path.join(pooled_data_dir, 'Results/Multivariate_Analysis')

### ComBat experiments for Diagnosis, Sex, and site classification 

In [ ]:
class_label = 'Diagnosis'

# For now only look into pairwise ROI-to-ROI FC
X = pairwise_ROI_to_ROI_FC_vec.copy()

# Only want to look into subjects that have Age and Sex covariates available. Not needed here
has_cov_mask = [i not in IDs_missing_age_and_sex_covars for i in included_subject_ids]
no_med_hc_mask = ~is_med_hc # We do NOT want to include any medicated HC 
subject_mask = merge_bool_masks(has_cov_mask, no_med_hc_mask)
assert sum(~subject_mask) == 0

# Filter out sites that have too little examples per class
site_mask, counts_df = has_N_samples_per_class(included_subjects_covariates_df, class_label, subject_mask, 
                                               N_threshold=10, verbose=True)

combined_mask = merge_bool_masks(subject_mask, site_mask)

y_Dx_masked = Dx_vec[combined_mask]
y_Sex_masked = Sex_vec[combined_mask]
y_Site_masked = LabelEncoder().fit_transform(groups_vec)[combined_mask]
groups_masked = groups_vec[combined_mask]
X_masked = X[combined_mask, :]

# Create EO = matrix of effects of interest to keep, (age, sex, diagnosis)
# EO = np.c_[demographic_data[combined_mask, :], y_Dx_masked]
EO = demographic_data[combined_mask, :]

Excluded 96 subjects belonging to 5 different sites:

Zurich_UCH
Dresden
Amsterdam_AMC
Kyushu
Barcelona_Bellvitge/COMPULSE_3T


In [65]:
def run_binary_clf_iter_with_combat_2023(X, y, EO, groups, clf_model, train_idx, test_idx):
    combat = Combat()
    scoring = Evaluater()

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    groups_train, groups_test = groups[train_idx], groups[test_idx]
    EO_train, EO_test = EO[train_idx], EO[test_idx]

    X_train = combat.fit_transform(Y=X_train, b=groups_train, X=EO_train, C=None)
    X_test = combat.transform(Y=X_test, b=groups_test, X=EO_test, C=None)

    clf_model.fit(X_train, y_train)

    y_pred = clf_model.predict(X_test)
    y_score = clf_model.decision_function(X_test)

    return scoring.evaluate_prediction(y_score=y_score,
                                       y_pred=y_pred,
                                       y_true=y_test)


def run_multi_clf_iter_with_combat_2023(X, y, y_Dx, EO, groups, clf_model, train_idx, test_idx):
    combat = Combat()

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    groups_train, groups_test = groups[train_idx], groups[test_idx]
    EO_train, EO_test = EO[train_idx], EO[test_idx]

    y_Dx_train, y_Dx_test = y_Dx[train_idx], y_Dx[test_idx]

    X_train = combat.fit_transform(Y=X_train, b=groups_train, X=EO_train, C=None)
    X_test = combat.transform(Y=X_test, b=groups_test, X=EO_test, C=None)

    clf_model.fit(X_train, y_train)
    y_pred = clf_model.predict(X_test)

    # Calculate and save multiclass metrics
    multiclass_report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
    multiclass_metrics = np.concatenate((multiclass_report_df.loc[['accuracy'], 'precision'].values,
                                         multiclass_report_df.loc['macro avg'].values,
                                         multiclass_report_df.loc['weighted avg'].values))

    return multiclass_report_df, multiclass_metrics


def run_binary_clf_ComBat_experiment_2023(y_label, X, y, EO, groups, cv_splits, save_dir, n_jobs=20):
    n_splits = len(cv_splits)

    # Set up scoring evaluation
    scoring = Evaluater()
    metric_labels = scoring.evaluate_labels()

    # Dummy classifier that uses y_train to create priors for predictions
    dummy_clf = DummyClassifier(strategy='stratified', random_state=0)

    # Set up linear SVM with precomputed Kernel
    K = pairwise_kernels(X, metric='linear')
    svm = SVC(kernel='precomputed', class_weight='balanced', C=1, probability=False)
    svm_pipe = Pipeline([('centerer', KernelCenterer()),
                         ('clf', svm)])

    metric_scores_svm = np.zeros((n_splits, len(metric_labels)))
    metric_scores_dummy = np.zeros((n_splits, len(metric_labels)))

    for i_split, (train_idx, test_idx) in enumerate(cv_splits):
        K_train, K_test = K[np.ix_(train_idx, train_idx)], K[np.ix_(test_idx, train_idx)]
        y_train, y_test = y[train_idx], y[test_idx]

        svm_pipe.fit(K_train, y_train)
        dummy_clf.fit(K_train, y_train)

        y_score_svm = svm_pipe.decision_function(K_test)
        y_pred_svm = svm_pipe.predict(K_test)

        y_pred_dummy = dummy_clf.predict(K_test)

        # Calculate and save metrics
        metric_scores_svm[i_split] = scoring.evaluate_prediction(y_score=y_score_svm, y_pred=y_pred_svm,
                                                                 y_true=y_test)

        metric_scores_dummy[i_split] = scoring.evaluate_prediction(y_score=y_pred_dummy, y_pred=y_pred_dummy,
                                                                   y_true=y_test)

    print("Dummy classifier performance:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores_dummy[:, m_idx].mean()))
    print("")

    print("SVM performance without ComBat:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores_svm[:, m_idx].mean()))
    print("")

    # Now run ComBat!
    svm_combat = SVC(kernel='linear', class_weight='balanced', C=1, probability=False)
    svm_pipe_combat = Pipeline([('centerer', StandardScaler(with_std=False)),
                                ('clf', svm_combat)])

    metric_scores_svm_combat = Parallel(n_jobs=n_jobs, verbose=1)(delayed(run_binary_clf_iter_with_combat_2023)
                                                                  (X, y, EO, groups, svm_pipe_combat, train_idx, test_idx)
                                                                  for i_split, (train_idx, test_idx)
                                                                  in enumerate(cv_splits))
    metric_scores_svm_combat = np.array(metric_scores_svm_combat, dtype=object)

    print("SVM performance with ComBat:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores_svm_combat[:, m_idx].mean()))
    print("")

    # Save results
    save_path = os.path.join(save_dir, 'results_' + y_label + '_clf.npz')

    np.savez(save_path,
             y_label=y_label,
             X=X, y=y, groups=groups,
             metric_scores_dummy=metric_scores_dummy,
             metric_scores_svm=metric_scores_svm,
             metric_scores_svm_combat=metric_scores_svm_combat,
             cv_splits=cv_splits,
             )

    return metric_scores_dummy, metric_scores_svm, metric_scores_svm_combat, metric_labels


def run_multi_clf_ComBat_experiment_2023(y_label, X, y, y_Dx, EO, groups, cv_splits, save_dir, n_jobs=20):
    n_splits = len(cv_splits)

    # Dummy classifier that uses y_train to create priors for predictions
    dummy_clf = DummyClassifier(strategy='stratified', random_state=0)

    # Set up SVM
    K = pairwise_kernels(X, metric='linear')
    svm = SVC(kernel='precomputed', class_weight='balanced', C=1, probability=False, decision_function_shape='ovr')
    svm_pipe = Pipeline([('centerer', KernelCenterer()),
                         ('clf', svm)])

    multiclass_metric_labels = ['accuracy', 'macro_avg_precision', 'macro_avg_recall', 'macro_avg_f1_score',
                                'macro_avg_support', 'weigh_avg_precision', 'weigh_avg_recall', 'weigh_avg_f1_score',
                                'weigh_avg_support']

    multiclass_metrics_svm = np.zeros((n_splits, len(multiclass_metric_labels)))
    multiclass_metrics_dummy = np.zeros((n_splits, len(multiclass_metric_labels)))

    for i_split, (train_idx, test_idx) in enumerate(cv_splits):
        K_train, K_test = K[np.ix_(train_idx, train_idx)], K[np.ix_(test_idx, train_idx)]
        y_train, y_test = y[train_idx], y[test_idx]

        svm_pipe.fit(K_train, y_train)
        dummy_clf.fit(K_train, y_train)

        y_pred_svm = svm_pipe.predict(K_test)
        y_pred_dummy = dummy_clf.predict(K_test)

        # Calculate and save multiclass metrics
        report_svm = pd.DataFrame(classification_report(y_test, y_pred_svm, output_dict=True)).transpose()
        report_dummy = pd.DataFrame(classification_report(y_test, y_pred_dummy, output_dict=True)).transpose()

        multiclass_metrics_svm_ = np.concatenate((report_svm.loc[['accuracy'], 'precision'].values,
                                                  report_svm.loc['macro avg'].values,
                                                  report_svm.loc['weighted avg'].values))
        multiclass_metrics_dummy_ = np.concatenate((report_dummy.loc[['accuracy'], 'precision'].values,
                                                    report_dummy.loc['macro avg'].values,
                                                    report_dummy.loc['weighted avg'].values))

        multiclass_metrics_svm[i_split, :] = multiclass_metrics_svm_
        multiclass_metrics_dummy[i_split, :] = multiclass_metrics_dummy_

    print("Dummy classifier performance:")
    for m_idx, m_label in enumerate(multiclass_metric_labels):
        print("mean {}: {:.2f}".format(m_label.capitalize(), multiclass_metrics_dummy[:, m_idx].mean()))
    print("")

    print("SVM performance without ComBat:")
    for m_idx, m_label in enumerate(multiclass_metric_labels):
        print("mean {}: {:.2f}".format(m_label.capitalize(), multiclass_metrics_svm[:, m_idx].mean()))
    print("")

    svm_combat = SVC(kernel='linear', class_weight='balanced', C=1, probability=False)
    svm_pipe_combat = Pipeline([('centerer', StandardScaler(with_std=False)),
                                ('clf', svm_combat)])

    multiclass_report_df_svm_combat, multiclass_metrics_svm_combat = zip(*Parallel(n_jobs=n_jobs, verbose=1)
    (delayed(run_multi_clf_iter_with_combat_2023)(X, y, y_Dx, EO,
                                             groups,
                                             svm_pipe_combat,
                                             train_idx,
                                             test_idx)
     for i_split, (train_idx, test_idx) in enumerate(cv_splits)))

    multiclass_metrics_svm_combat = np.array(multiclass_metrics_svm_combat, dtype=object)

    print("SVM performance with ComBat:")
    for m_idx, m_label in enumerate(multiclass_metric_labels):
        print("mean {}: {:.2f}".format(m_label.capitalize(), multiclass_metrics_svm_combat[:, m_idx].mean()))
    print("")

    # Save results
    save_path = os.path.join(save_dir, 'results_' + y_label + '_clf.npz')

    np.savez(save_path,
             y_label=y_label,
             X=X, y=y, groups=groups,
             multiclass_metrics_dummy=multiclass_metrics_dummy,
             multiclass_metrics_svm=multiclass_metrics_svm,
             multiclass_metrics_svm_combat=multiclass_metrics_svm_combat,
             cv_splits=cv_splits,
             )

    return multiclass_metrics_dummy, multiclass_metrics_svm, multiclass_metrics_svm_combat, multiclass_metric_labels


In [ ]:
# For these ComBat experiments we want to ensure same subjects are used for Dx, Sex and Site classification

outer_cv_folds, outer_cv_repeats = 5, 20
inner_cv_folds, inner_cv_repeats = 5, 1

class_label = 'Diagnosis'

combat_save_dir = os.path.join(CLF_RESULTS_DIR, 'ComBat_experiments_2023_EO_AgeSex')
ensure_folder(combat_save_dir)

# For now only look into pairwise ROI-to-ROI FC
X = pairwise_ROI_to_ROI_FC_vec.copy()

# Only want to look into subjects that have Age and Sex covariates available. Not needed here
has_cov_mask = [i not in IDs_missing_age_and_sex_covars for i in included_subject_ids]
no_med_hc_mask = ~is_med_hc # We do NOT want to include any medicated HC 
subject_mask = merge_bool_masks(has_cov_mask, no_med_hc_mask)
assert sum(~subject_mask) == 0

# Filter out sites that have too little examples per class
site_mask, counts_df = has_N_samples_per_class(included_subjects_covariates_df, class_label, subject_mask, 
                                               N_threshold=10, verbose=True)

combined_mask = merge_bool_masks(subject_mask, site_mask)

y_Dx_masked = Dx_vec[combined_mask]
y_Sex_masked = Sex_vec[combined_mask]
y_Site_masked = LabelEncoder().fit_transform(groups_vec)[combined_mask]
groups_masked = groups_vec[combined_mask]
X_masked = X[combined_mask, :]

# Make sure no NaNs are present in any of the labels to predict
for v in [y_Dx_masked, y_Sex_masked, y_Site_masked]:
    assert np.sum(np.isnan(v)) == 0

    
# Ensure we use same CV splits across different classification tasks: stratified for Site x Diagnosis 
cv_splits = repeated_group_stratified_KFold_splits(y_Dx_masked, groups_masked, 
                                                   n_folds=outer_cv_folds, n_repeats=outer_cv_repeats, 
                                                   seed=0, verbose=True)
n_splits=len(cv_splits)

    
# Experiment 1: What is Dx clf performance for Dummy classifier, SVM without ComBat, and with ComBat?
y_label = 'Diagnosis'
metric_scores_dummy, metric_scores_svm, metric_scores_svm_combat, metric_labels = run_binary_clf_ComBat_experiment_2023(
                                                                                                    y_label,
                                                                                                    X_masked, 
                                                                                                    y_Dx_masked,
                                                                                                    EO,
                                                                                                    groups_masked, 
                                                                                                    cv_splits,
                                                                                                    combat_save_dir)
# Plot classification results
plot_ComBat_experiment_results(y_label, metric_scores_dummy, metric_scores_svm, metric_scores_svm_combat,
                               metric_labels, combat_save_dir)


# Experiment 2: What is Dx clf performance for Dummy classifier, SVM without ComBat, and with ComBat?
y_label = 'Sex'
metric_scores_dummy, metric_scores_svm, metric_scores_svm_combat, metric_labels = run_binary_clf_ComBat_experiment_2023(
                                                                                                    y_label,
                                                                                                    X_masked, 
                                                                                                    y_Sex_masked,
                                                                                                    EO,
                                                                                                    groups_masked, 
                                                                                                    cv_splits,
                                                                                                    combat_save_dir)
# Plot classification results
plot_ComBat_experiment_results(y_label, metric_scores_dummy, metric_scores_svm, metric_scores_svm_combat,
                               metric_labels, combat_save_dir)


# Experiment 3: What is Site (multi-)clf performance for Dummy classifier, SVM without ComBat, and with ComBat?
y_label = 'Site'
(multiclass_metrics_dummy, multiclass_metrics_svm, 
multiclass_metrics_svm_combat, multiclass_metric_labels) = run_multi_clf_ComBat_experiment_2023(y_label,
                                                                                                X_masked, 
                                                                                                y_Site_masked,
                                                                                                y_Dx_masked,
                                                                                                EO,
                                                                                                groups_masked, 
                                                                                                cv_splits,
                                                                                                combat_save_dir)
# Multi site chance level performance is 1/N sites
multisite_y_chance = float(1) / len(np.unique(y_Site_masked))

# Plot classification results
plot_ComBat_experiment_results(y_label, multiclass_metrics_dummy, multiclass_metrics_svm, 
                               multiclass_metrics_svm_combat, multiclass_metric_labels, combat_save_dir,
                               y_chance=multisite_y_chance, multiclass=True)

Excluded 96 subjects belonging to 5 different sites:

Zurich_UCH
Dresden
Amsterdam_AMC
Kyushu
Barcelona_Bellvitge/COMPULSE_3T
Dummy classifier performance:
mean accuracy: 0.49
mean balanced_accuracy: 0.49
mean AUC: 0.49
mean F1: 0.49
mean recall: 0.49
mean precision: 0.49
mean sensitivity: 0.49
mean specificity: 0.49
mean positive_predictive_value: 0.49
mean negative_predictive_value: 0.49

SVM performance without ComBat:
mean accuracy: 0.61
mean balanced_accuracy: 0.61
mean AUC: 0.66
mean F1: 0.61
mean recall: 0.62
mean precision: 0.61
mean sensitivity: 0.62
mean specificity: 0.60
mean positive_predictive_value: 0.61
mean negative_predictive_value: 0.61



[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:  5.2min
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed: 20.7min finished


SVM performance with ComBat:
mean accuracy: 0.59
mean balanced_accuracy: 0.59
mean AUC: 0.63
mean F1: 0.59
mean recall: 0.60
mean precision: 0.59
mean sensitivity: 0.60
mean specificity: 0.59
mean positive_predictive_value: 0.59
mean negative_predictive_value: 0.59



/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


Dummy classifier performance:
mean accuracy: 0.50
mean balanced_accuracy: 0.50
mean AUC: 0.50
mean F1: 0.50
mean recall: 0.50
mean precision: 0.50
mean sensitivity: 0.50
mean specificity: 0.50
mean positive_predictive_value: 0.50
mean negative_predictive_value: 0.50

SVM performance without ComBat:
mean accuracy: 0.75
mean balanced_accuracy: 0.75
mean AUC: 0.83
mean F1: 0.74
mean recall: 0.73
mean precision: 0.76
mean sensitivity: 0.73
mean specificity: 0.78
mean positive_predictive_value: 0.76
mean negative_predictive_value: 0.74



...

/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

Dummy classifier performance:
mean Accuracy: 0.06
mean Macro_avg_precision: 0.03
mean Macro_avg_recall: 0.03
mean Macro_avg_f1_score: 0.03
mean Macro_avg_support: 410.40
mean Weigh_avg_precision: 0.07
mean Weigh_avg_recall: 0.06
mean Weigh_avg_f1_score: 0.07
mean Weigh_avg_support: 410.40

SVM performance without ComBat:
mean Accuracy: 0.73
mean Macro_avg_precision: 0.73
mean Macro_avg_recall: 0.64
mean Macro_avg_f1_score: 0.66
mean Macro_avg_support: 410.40
mean Weigh_avg_precision: 0.74
mean Weigh_avg_recall: 0.73
mean Weigh_avg_f1_score: 0.71
mean Weigh_avg_support: 410.40



[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted sa

SVM performance with ComBat:
mean Accuracy: 0.03
mean Macro_avg_precision: 0.03
mean Macro_avg_recall: 0.02
mean Macro_avg_f1_score: 0.02
mean Macro_avg_support: 410.40
mean Weigh_avg_precision: 0.07
mean Weigh_avg_recall: 0.03
mean Weigh_avg_f1_score: 0.04
mean Weigh_avg_support: 410.40



/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [44]:
def run_gridsearch(X, y, pipeline, grid_params, cv_splits, n_jobs, scoring='roc_auc', verbose=False):
    
    grid = GridSearchCV(pipeline, param_grid=grid_params, cv=cv_splits, verbose=verbose, scoring=scoring, 
                        n_jobs=n_jobs, pre_dispatch='2*n_jobs', refit=True)
    grid.fit(X, y)
    
    best_params = [grid.best_params_[param] for param in sorted(grid.best_params_)]
    
    return grid.best_estimator_, best_params, grid.best_score_

In [45]:
# Initialise gridsearch params which are identical for all classifications

C_range = np.logspace(-3, 1, 5) # [0.001, 0.01, 0.1, 1.0, 10]
grid_params_lin_K = {'clf__C' : C_range} 

# gamma_range = np.logspace(-4, 0, 5) # [0.0001, 0.001, 0.01, 0.1, 1.0]
# grid_params_rbf_K = {'clf__C' : C_range,
#                      'clf__gamma' : gamma_range} 

In [46]:
# We will use site and class stratified 5 Fold CV with 20 repeats

outer_cv_folds, outer_cv_repeats = 5, 20
inner_cv_folds, inner_cv_repeats = 5, 1

### Classification seperately for pediatric (<18 years) and adult (>= 18 years) samples

In [48]:
RESULTS_DIR = os.path.join(CLF_RESULTS_DIR, '1_Main_Analyses')

N_jobs = 20 

title_template = '{}_CLF_{}_SAMPLES_{}_FEATURES'
class_label = 'Diagnosis'
y = Dx_vec.copy()

# All analysis-specific subject masking can be defined BEFORE looping through age groups and feature sets:
has_y_mask = ~np.isnan(y) # Redundant here, but we'll need it for Med/Sev/AO classifications

# Now iterate over all age groups and feature sets
for age_group, feature_label in product(age_groups, feature_labels): 
    
    title = title_template.format(class_label, age_group, feature_label)
    print(title)
    
    save_dir = os.path.join(RESULTS_DIR, title)
    
    if os.path.exists(save_dir):
        print("Output directory already exists, skipping...")
        continue
    
    ensure_folder(save_dir)
    
    X = feature_sets_dict[feature_label]
    
    # Merge subject masks
    age_group_mask = age_group_masks_dict[age_group]
    subject_mask = merge_bool_masks(has_y_mask, age_group_mask)
    
    # Create copy of covariates df and append class_label vec is this is not included already
    tmp_covariates_df = included_subjects_covariates_df.copy()
    if class_label not in tmp_covariates_df.columns:
        tmp_covariates_df[class_label] = y
        
    # Filter out sites that have too little examples per class
    site_mask, counts_df = has_N_samples_per_class(tmp_covariates_df, class_label, subject_mask, 
                                                   N_threshold=10, verbose=True)
    
    # Save overview of included sites and class counts
    counts_df.to_csv(os.path.join(save_dir, 'class_counts.csv'), index=False)
    
    if counts_df.shape[0] < 2:
        print("Not enough samples to run analysis. Skipping!")
        continue
    
    # Combine analysis-specific subject mask with site mask
    combined_mask = merge_bool_masks(subject_mask, site_mask)
    
    # Apply combined mask
    y_, groups_ = y[combined_mask], groups_vec[combined_mask]
    X_ = X[combined_mask, :]
    
    # Create CV splits
    cv_splits = repeated_group_stratified_KFold_splits(y_, groups_, 
                                                       n_folds=outer_cv_folds, n_repeats=outer_cv_repeats, 
                                                       seed=0, verbose=True)
    n_splits=len(cv_splits)

    # Initialize scoring evaluaters and classification pipeline
    scoring = Evaluater()
    metric_labels = scoring.evaluate_labels()
    metric_scores = np.zeros((n_splits, len(metric_labels)))

    best_params_lin = np.zeros((n_splits))
    
    y_predictions = np.ones((n_splits, len(y_))) * -1
    y_scores = np.ones((n_splits, len(y_))) * -1

    svm = SVC(kernel='precomputed', class_weight='balanced', C=1)
    svm_pipe = Pipeline([('centerer', KernelCenterer()), 
                         ('clf', svm)])

    # Precompute Kernels
    K_lin = pairwise_kernels(X_, metric='linear')
    
    t1 = time()    
    
    # Run cross-validation
    for i_split, (train_idx, test_idx) in enumerate(cv_splits):

        t3 = time()

        K_lin_train, K_lin_test = K_lin[np.ix_(train_idx, train_idx)], K_lin[np.ix_(test_idx, train_idx)]   
        y_train, y_test = y_[train_idx], y_[test_idx]
        groups_train = groups_[train_idx]

        nested_cv_splits = repeated_group_stratified_KFold_splits(y_train, groups_train, 
                                                                  n_folds=inner_cv_folds, n_repeats=inner_cv_repeats,
                                                                  seed=0, verbose=True)

        # Run gridsearch for linear Kernel (C)
        lin_model, lin_best_params, lin_best_score = run_gridsearch(K_lin_train, y_train, 
                                                                    svm_pipe, grid_params_lin_K, 
                                                                    nested_cv_splits, n_jobs=20, 
                                                                    scoring='balanced_accuracy', verbose=True)
        y_score = lin_model.decision_function(K_lin_test) 
        y_pred = lin_model.predict(K_lin_test)
        metric_scores[i_split, :] = scoring.evaluate_prediction(y_score=y_score, y_pred=y_pred, y_true=y_test)
        best_params_lin[i_split] = lin_best_params[0]
        y_predictions[i_split, test_idx] = y_pred
        y_scores[i_split, test_idx] = y_score

        t4 = time()

        print("Finished CV iteration {}/{} in {:.2f} minutes: ".format(i_split+1, n_splits, (t4-t3)/60.))

    t2 = time() 
    
    print()
    print("Finished classification in {:.2f} minutes".format((t2-t1)/60.))
    print()
    
    print("Performance averaged across splits:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores[:, m_idx].mean()))
    print()
        
    # Save classification results
    np.savez(os.path.join(save_dir, 'clf_results.npz'), 
             cv_splits=cv_splits,
             metric_scores=metric_scores,
             metric_labels=metric_labels,
             y_predictions=y_predictions,
             y_scores=y_scores,
             best_params_lin=best_params_lin,
             dtype=object
            )
    
    # Save results summary to DataFrame for quick overview
    results_df = pd.DataFrame([metric_scores.mean(axis=0)], columns=metric_labels)
    results_df.to_csv(os.path.join(save_dir, 'clf_summary.csv'))

Diagnosis_CLF_pooled_SAMPLES_demographical_FEATURES
ERROR! Session/line number was not unique in database. History logging moved to new session 2893
Excluded 96 subjects belonging to 5 different sites:

Zurich_UCH
Dresden
Amsterdam_AMC
Kyushu
Barcelona_Bellvitge/COMPULSE_3T
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 1/100 in 0.80 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 2/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 3/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 4/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 5/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 6/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 7/100 i

Finished CV iteration 75/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 76/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 77/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 78/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 79/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 80/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 81/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 82/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 83/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 84/100 in 0.01 

/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


Finished CV iteration 1/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 2/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 3/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 4/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 5/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 6/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 7/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 8/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 9/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 10/100 in 0.00 minutes: 

Finished CV iteration 78/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 79/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 80/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 81/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 82/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 83/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 84/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 85/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 86/100 in 0.01 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 87/100 in 0.01 

...

/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/notebooks/scripts/evaluation_classifier.py:13: RuntimeWarning: invalid value encountered in true_divide
  ppv_score = np.sum((y_true == 1) & (y_pred == 1)) / np.sum(y_pred == 1, dtype=np.float)
/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/notebooks/scripts/evaluation_classifier.py:13: RuntimeWarning: i

Finished CV iteration 85/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 86/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 87/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 88/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 89/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 90/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits


/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/notebooks/scripts/evaluation_classifier.py:13: RuntimeWarning: invalid value encountered in true_divide
  ppv_score = np.sum((y_true == 1) & (y_pred == 1)) / np.sum(y_pred == 1, dtype=np.float)
/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/notebooks/scripts/evaluation_classifier.py:13: RuntimeWarning: i

Finished CV iteration 91/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 92/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 93/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 94/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 95/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 96/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 97/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 98/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 99/100 in 0.00 minutes: 
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Finished CV iteration 100/100 in 0.00

/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/notebooks/scripts/evaluation_classifier.py:13: RuntimeWarning: invalid value encountered in true_divide
  ppv_score = np.sum((y_true == 1) & (y_pred == 1)) / np.sum(y_pred == 1, dtype=np.float)
/data/wbbruin/anaconda2/envs/ENIGMA_OCD_RSFMRI/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/wbbruin/Desktop/ENIGMA_OCD_RSFMRI_2021/notebooks/scripts/evaluation_classifier.py:13: RuntimeWarning: i

In [ ]:
# 2a) Medicated / Unmedicated OCD vs Controls

RESULTS_DIR = os.path.join(CLF_RESULTS_DIR, '2a_Medication_Analyses')

N_jobs = 20 

title_template = '{}_CLF_{}_SAMPLES_{}_FEATURES_{}_FILTER'
class_label = 'Diagnosis'
y = Dx_vec.copy()

# All analysis-specific subject masking can be defined BEFORE looping through age groups and feature sets:

# Only want to look into subjects that have Age and Sex covariates available
has_y_mask = ~np.isnan(y) # Redundant here, but we'll need it for Med/Sev/AO classifacations

# Now iterate over all age groups and feature sets
for age_group, feature_label, filter_label in product(age_groups, feature_labels, Dx_med_filter_labels):
    
    title = title_template.format(class_label, age_group, feature_label, filter_label)
    print(title)
    
    save_dir = os.path.join(RESULTS_DIR, title)
    
    if os.path.exists(save_dir):
        print("Output directory already exists, skipping...")
        continue
    
    ensure_folder(save_dir)
    
    X = feature_sets_dict[feature_label]
    
    # Merge subject masks
    age_group_mask = age_group_masks_dict[age_group]
    filter_mask = Dx_med_filter_dict[filter_label]
    subject_mask = merge_bool_masks(has_y_mask, age_group_mask, filter_mask)
    
    # Create copy of covariates df and append class_label vec is this is not included already
    tmp_covariates_df = included_subjects_covariates_df.copy()
    if class_label not in tmp_covariates_df.columns:
        tmp_covariates_df[class_label] = y
        
    # Filter out sites that have too little examples per class
    site_mask, counts_df = has_N_samples_per_class(tmp_covariates_df, class_label, subject_mask, 
                                                   N_threshold=10, verbose=True)
    
    # Save overview of included sites and class counts
    counts_df.to_csv(os.path.join(save_dir, 'class_counts.csv'), index=False)
    
    if counts_df.shape[0] < 2:
        print("Not enough samples to run analysis. Skipping!")
        continue
    
    # Combine analysis-specific subject mask with site mask
    combined_mask = merge_bool_masks(subject_mask, site_mask)
    
    # Apply combined mask
    y_, groups_ = y[combined_mask], groups_vec[combined_mask]
    X_ = X[combined_mask, :]
    
    # Create CV splits
    cv_splits = repeated_group_stratified_KFold_splits(y_, groups_, 
                                                       n_folds=outer_cv_folds, n_repeats=outer_cv_repeats, 
                                                       seed=0, verbose=True)
    n_splits=len(cv_splits)

    # Initialize scoring evaluaters and classification pipeline
    scoring = Evaluater()
    metric_labels = scoring.evaluate_labels()
    metric_scores = np.zeros((n_splits, len(metric_labels)))

    best_params_lin = np.zeros((n_splits))
    
    y_predictions = np.ones((n_splits, len(y_))) * -1
    y_scores = np.ones((n_splits, len(y_))) * -1

    svm = SVC(kernel='precomputed', class_weight='balanced', C=1)
    svm_pipe = Pipeline([('centerer', KernelCenterer()), 
                         ('clf', svm)])

    # Precompute Kernels
    K_lin = pairwise_kernels(X_, metric='linear')
    
    t1 = time()    
    
    # Run cross-validation
    for i_split, (train_idx, test_idx) in enumerate(cv_splits):

        t3 = time()

        K_lin_train, K_lin_test = K_lin[np.ix_(train_idx, train_idx)], K_lin[np.ix_(test_idx, train_idx)]   
        y_train, y_test = y_[train_idx], y_[test_idx]
        groups_train = groups_[train_idx]

        nested_cv_splits = repeated_group_stratified_KFold_splits(y_train, groups_train, 
                                                                  n_folds=inner_cv_folds, n_repeats=inner_cv_repeats,
                                                                  seed=0, verbose=True)

        # Run gridsearch for linear Kernel (C)
        lin_model, lin_best_params, lin_best_score = run_gridsearch(K_lin_train, y_train, 
                                                                    svm_pipe, grid_params_lin_K, 
                                                                    nested_cv_splits, n_jobs=20, 
                                                                    scoring='balanced_accuracy', verbose=True)
        y_score = lin_model.decision_function(K_lin_test) 
        y_pred = lin_model.predict(K_lin_test)
        metric_scores[i_split, :] = scoring.evaluate_prediction(y_score=y_score, y_pred=y_pred, y_true=y_test)
        best_params_lin[i_split] = lin_best_params[0]
        y_predictions[i_split, test_idx] = y_pred
        y_scores[i_split, test_idx] = y_score

        t4 = time()

        print("Finished CV iteration {}/{} in {:.2f} minutes: ".format(i_split+1, n_splits, (t4-t3)/60.))

    t2 = time() 
    
    print()
    print("Finished classification in {:.2f} minutes".format((t2-t1)/60.))
    print()
    
    print("Performance averaged across splits:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores[:, m_idx].mean()))
    print()
        
    # Save classification results
    np.savez(os.path.join(save_dir, 'clf_results.npz'), 
             cv_splits=cv_splits,
             metric_scores=metric_scores,
             metric_labels=metric_labels,
             y_predictions=y_predictions,
             y_scores=y_scores,
             best_params_lin=best_params_lin,
             dtype=object
            )
    
    # Save results summary to DataFrame for quick overview
    results_df = pd.DataFrame([metric_scores.mean(axis=0)], columns=metric_labels)
    results_df.to_csv(os.path.join(save_dir, 'clf_summary.csv'))

In [ ]:
# 2b) Early / Late Onset OCD vs Controls

RESULTS_DIR = os.path.join(CLF_RESULTS_DIR, '2b_Age_of_Onset_Analyses')

N_jobs = 20 

title_template = '{}_CLF_{}_SAMPLES_{}_FEATURES_{}_FILTER'
class_label = 'Diagnosis'
y = Dx_vec.copy()

# All analysis-specific subject masking can be defined BEFORE looping through age groups and feature sets:
has_y_mask = ~np.isnan(y) # Redundant here, but we'll need it for Med/Sev/AO classifacations

# Now iterate over all age groups and feature sets
for age_group, feature_label, filter_label in product(age_groups, feature_labels, Dx_AO_filter_labels):
    
    title = title_template.format(class_label, age_group, feature_label, filter_label)
    print(title)
    
    save_dir = os.path.join(RESULTS_DIR, title)
    
    if os.path.exists(save_dir):
        print("Output directory already exists, skipping...")
        continue
    
    ensure_folder(save_dir)
    
    X = feature_sets_dict[feature_label]
    
    # Merge subject masks
    age_group_mask = age_group_masks_dict[age_group]
    filter_mask = Dx_AO_filter_dict[filter_label]
    subject_mask = merge_bool_masks(has_y_mask, age_group_mask, filter_mask)
    
    # Create copy of covariates df and append class_label vec is this is not included already
    tmp_covariates_df = included_subjects_covariates_df.copy()
    if class_label not in tmp_covariates_df.columns:
        tmp_covariates_df[class_label] = y
        
    # Filter out sites that have too little examples per class
    site_mask, counts_df = has_N_samples_per_class(tmp_covariates_df, class_label, subject_mask, 
                                                   N_threshold=10, verbose=True)
    
    # Save overview of included sites and class counts
    counts_df.to_csv(os.path.join(save_dir, 'class_counts.csv'), index=False)
    
    if counts_df.shape[0] < 2:
        print("Not enough samples to run analysis. Skipping!")
        continue
    
    # Combine analysis-specific subject mask with site mask
    combined_mask = merge_bool_masks(subject_mask, site_mask)
    
    # Apply combined mask
    y_, groups_ = y[combined_mask], groups_vec[combined_mask]
    X_ = X[combined_mask, :]
    
    # Create CV splits
    cv_splits = repeated_group_stratified_KFold_splits(y_, groups_, 
                                                       n_folds=outer_cv_folds, n_repeats=outer_cv_repeats, 
                                                       seed=0, verbose=True)
    n_splits=len(cv_splits)

    # Initialize scoring evaluaters and classification pipeline
    scoring = Evaluater()
    metric_labels = scoring.evaluate_labels()
    metric_scores = np.zeros((n_splits, len(metric_labels)))

    best_params_lin = np.zeros((n_splits))
    
    y_predictions = np.ones((n_splits, len(y_))) * -1
    y_scores = np.ones((n_splits, len(y_))) * -1

    svm = SVC(kernel='precomputed', class_weight='balanced', C=1)
    svm_pipe = Pipeline([('centerer', KernelCenterer()), 
                         ('clf', svm)])

    # Precompute Kernels
    K_lin = pairwise_kernels(X_, metric='linear')
    
    t1 = time()    
    
    # Run cross-validation
    for i_split, (train_idx, test_idx) in enumerate(cv_splits):

        t3 = time()

        K_lin_train, K_lin_test = K_lin[np.ix_(train_idx, train_idx)], K_lin[np.ix_(test_idx, train_idx)]   
        y_train, y_test = y_[train_idx], y_[test_idx]
        groups_train = groups_[train_idx]

        nested_cv_splits = repeated_group_stratified_KFold_splits(y_train, groups_train, 
                                                                  n_folds=inner_cv_folds, n_repeats=inner_cv_repeats,
                                                                  seed=0, verbose=True)

        # Run gridsearch for linear Kernel (C)
        lin_model, lin_best_params, lin_best_score = run_gridsearch(K_lin_train, y_train, 
                                                                    svm_pipe, grid_params_lin_K, 
                                                                    nested_cv_splits, n_jobs=20, 
                                                                    scoring='balanced_accuracy', verbose=True)
        y_score = lin_model.decision_function(K_lin_test) 
        y_pred = lin_model.predict(K_lin_test)
        metric_scores[i_split, :] = scoring.evaluate_prediction(y_score=y_score, y_pred=y_pred, y_true=y_test)
        best_params_lin[i_split] = lin_best_params[0]
        y_predictions[i_split, test_idx] = y_pred
        y_scores[i_split, test_idx] = y_score

        t4 = time()

        print("Finished CV iteration {}/{} in {:.2f} minutes: ".format(i_split+1, n_splits, (t4-t3)/60.))

    t2 = time() 
    
    print()
    print("Finished classification in {:.2f} minutes".format((t2-t1)/60.))
    print()
    
    print("Performance averaged across splits:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores[:, m_idx].mean()))
    print()
        
    # Save classification results
    np.savez(os.path.join(save_dir, 'clf_results.npz'), 
             cv_splits=cv_splits,
             metric_scores=metric_scores,
             metric_labels=metric_labels,
             y_predictions=y_predictions,
             y_scores=y_scores,
             best_params_lin=best_params_lin,
             dtype=object
            )
    
    # Save results summary to DataFrame for quick overview
    results_df = pd.DataFrame([metric_scores.mean(axis=0)], columns=metric_labels)
    results_df.to_csv(os.path.join(save_dir, 'clf_summary.csv'))

In [ ]:
# 2c) Low / High Severity OCD vs Controls

RESULTS_DIR = os.path.join(CLF_RESULTS_DIR, '2c_Severity_Analyses')

N_jobs = 20 

title_template = '{}_CLF_{}_SAMPLES_{}_FEATURES_{}_FILTER'
class_label = 'Diagnosis'
y = Dx_vec.copy()

# All analysis-specific subject masking can be defined BEFORE looping through age groups and feature sets:
has_y_mask = ~np.isnan(y) # Redundant here, but we'll need it for Med/Sev/AO classifacations

# Now iterate over all age groups and feature sets
for age_group, feature_label, filter_label in product(age_groups, feature_labels, Dx_sev_filter_labels):
    
    title = title_template.format(class_label, age_group, feature_label, filter_label)
    print(title)
    
    save_dir = os.path.join(RESULTS_DIR, title)
    
    if os.path.exists(save_dir):
        print("Output directory already exists, skipping...")
        continue
    
    ensure_folder(save_dir)
    
    X = feature_sets_dict[feature_label]
    
    # Merge subject masks
    age_group_mask = age_group_masks_dict[age_group]
    filter_mask = Dx_sev_filter_dict[filter_label]
    subject_mask = merge_bool_masks(has_y_mask, age_group_mask, filter_mask)
    
    # Create copy of covariates df and append class_label vec is this is not included already
    tmp_covariates_df = included_subjects_covariates_df.copy()
    if class_label not in tmp_covariates_df.columns:
        tmp_covariates_df[class_label] = y
        
    # Filter out sites that have too little examples per class
    site_mask, counts_df = has_N_samples_per_class(tmp_covariates_df, class_label, subject_mask, 
                                                   N_threshold=10, verbose=True)
    
    # Save overview of included sites and class counts
    counts_df.to_csv(os.path.join(save_dir, 'class_counts.csv'), index=False)
    
    if counts_df.shape[0] < 2:
        print("Not enough samples to run analysis. Skipping!")
        continue
    
    # Combine analysis-specific subject mask with site mask
    combined_mask = merge_bool_masks(subject_mask, site_mask)
    
    # Apply combined mask
    y_, groups_ = y[combined_mask], groups_vec[combined_mask]
    X_ = X[combined_mask, :]
    
    # Create CV splits
    cv_splits = repeated_group_stratified_KFold_splits(y_, groups_, 
                                                       n_folds=outer_cv_folds, n_repeats=outer_cv_repeats, 
                                                       seed=0, verbose=True)
    n_splits=len(cv_splits)

    # Initialize scoring evaluaters and classification pipeline
    scoring = Evaluater()
    metric_labels = scoring.evaluate_labels()
    metric_scores = np.zeros((n_splits, len(metric_labels)))

    best_params_lin = np.zeros((n_splits))
    
    y_predictions = np.ones((n_splits, len(y_))) * -1
    y_scores = np.ones((n_splits, len(y_))) * -1

    svm = SVC(kernel='precomputed', class_weight='balanced', C=1)
    svm_pipe = Pipeline([('centerer', KernelCenterer()), 
                         ('clf', svm)])

    # Precompute Kernels
    K_lin = pairwise_kernels(X_, metric='linear')
    
    t1 = time()    
    
    # Run cross-validation
    for i_split, (train_idx, test_idx) in enumerate(cv_splits):

        t3 = time()

        K_lin_train, K_lin_test = K_lin[np.ix_(train_idx, train_idx)], K_lin[np.ix_(test_idx, train_idx)]   
        y_train, y_test = y_[train_idx], y_[test_idx]
        groups_train = groups_[train_idx]

        nested_cv_splits = repeated_group_stratified_KFold_splits(y_train, groups_train, 
                                                                  n_folds=inner_cv_folds, n_repeats=inner_cv_repeats,
                                                                  seed=0, verbose=True)

        # Run gridsearch for linear Kernel (C)
        lin_model, lin_best_params, lin_best_score = run_gridsearch(K_lin_train, y_train, 
                                                                    svm_pipe, grid_params_lin_K, 
                                                                    nested_cv_splits, n_jobs=20, 
                                                                    scoring='balanced_accuracy', verbose=True)
        y_score = lin_model.decision_function(K_lin_test) 
        y_pred = lin_model.predict(K_lin_test)
        metric_scores[i_split, :] = scoring.evaluate_prediction(y_score=y_score, y_pred=y_pred, y_true=y_test)
        best_params_lin[i_split] = lin_best_params[0]
        y_predictions[i_split, test_idx] = y_pred
        y_scores[i_split, test_idx] = y_score

        t4 = time()

        print("Finished CV iteration {}/{} in {:.2f} minutes: ".format(i_split+1, n_splits, (t4-t3)/60.))

    t2 = time() 
    
    print()
    print("Finished classification in {:.2f} minutes".format((t2-t1)/60.))
    print()
    
    print("Performance averaged across splits:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores[:, m_idx].mean()))
    print()
        
    # Save classification results
    np.savez(os.path.join(save_dir, 'clf_results.npz'), 
             cv_splits=cv_splits,
             metric_scores=metric_scores,
             metric_labels=metric_labels,
             y_predictions=y_predictions,
             y_scores=y_scores,
             best_params_lin=best_params_lin,
             dtype=object
            )
    
    # Save results summary to DataFrame for quick overview
    results_df = pd.DataFrame([metric_scores.mean(axis=0)], columns=metric_labels)
    results_df.to_csv(os.path.join(save_dir, 'clf_summary.csv'))

### 3) Classifications within subgroups of OCD patients ###

In [ ]:
# 3) Investigate effects of medication, AO and severity in patients only

is_med_pt_or_unmed_pt_mask = is_med_pt | is_unmed_pt 
is_early_AO_pt_or_late_AO_pt_mask = is_early_AO_pt | is_late_AO_pt
is_low_sev_pt_or_high_sev_pt_mask = is_low_sev_pt | is_high_sev_pt 

# "Medication status at time of scan (1=naive, 2=unmedicated, 3=medicated) recoded to Unmed = 0, Med = 1
assert included_subjects_covariates_df.loc[is_med_pt_or_unmed_pt_mask].Diagnosis.unique()[0] ==  1
med_vec = np.array(included_subjects_covariates_df.Medication_status)
med_vec = np.where(med_vec==1, 0, med_vec)
med_vec = np.where(med_vec==2, 0, med_vec)
med_vec = np.where(med_vec==3, 1, med_vec)
assert len(np.unique(med_vec[is_med_pt_or_unmed_pt_mask])) == 2

# Early-onset/Child OCD patients (onset before age 18) and Late-onset/adult OCD patients (onset at age 18 or older)
# Recode from 1=child, 2=adult to, 0 and 1 respectively
assert included_subjects_covariates_df.loc[is_early_AO_pt_or_late_AO_pt_mask].Diagnosis.unique()[0] ==  1
AO_vec = np.array(included_subjects_covariates_df.Age_of_onset)
AO_vec = np.where(AO_vec==1, 0, AO_vec)
AO_vec = np.where(AO_vec==2, 1, AO_vec)
assert len(np.unique(AO_vec[is_early_AO_pt_or_late_AO_pt_mask])) == 2

# (YBOCS <= 25; mild-moderate) and high severity (YBOCS > 25; moderate-severe) OCD; recode to Low=0, High=1
assert included_subjects_covariates_df.loc[is_low_sev_pt_or_high_sev_pt_mask].Diagnosis.unique()[0] ==  1
sev_vec = np.array(included_subjects_covariates_df['Y-BOCS'])
sev_vec[sev_vec <= median_sev] = 0
sev_vec[sev_vec > median_sev] = 1
assert len(np.unique(sev_vec[is_low_sev_pt_or_high_sev_pt_mask])) == 2

In [ ]:
# 3a) Medicated (coded=1) vs Unmedicated OCD (coded=0)
RESULTS_DIR = os.path.join(CLF_RESULTS_DIR, '3a_Medication_Analyses')

N_jobs = 20 

title_template = '{}_CLF_{}_SAMPLES_{}_FEATURES'

# Only have to change these 3 variables for different subgroup analyses.
filter_mask = is_med_pt_or_unmed_pt_mask # Specific for analyses: only look into patients subgroups    
class_label = 'Medication'
y = med_vec.copy()
                           
# Now iterate over all age groups and feature sets
for age_group, feature_label in product(age_groups, feature_labels):
    
    title = title_template.format(class_label, age_group, feature_label)
    print(title)
    
    save_dir = os.path.join(RESULTS_DIR, title)
    
    if os.path.exists(save_dir):
        print("Output directory already exists, skipping...")
        continue
    
    ensure_folder(save_dir)
    
    X = feature_sets_dict[feature_label]
    
    # Merge subject masks
    age_group_mask = age_group_masks_dict[age_group]
    subject_mask = merge_bool_masks(age_group_mask, filter_mask)
    
    # Create copy of covariates df and append class_label vec is this is not included already
    tmp_covariates_df = included_subjects_covariates_df.copy()
    if class_label not in tmp_covariates_df.columns:
        tmp_covariates_df[class_label] = y
        
    # Filter out sites that have too little examples per class
    site_mask, counts_df = has_N_samples_per_class(tmp_covariates_df, class_label, subject_mask, 
                                                   N_threshold=10, verbose=True)
    
    # Save overview of included sites and class counts
    counts_df.to_csv(os.path.join(save_dir, 'class_counts.csv'), index=False)
    
    if counts_df.shape[0] < 2:
        print("Not enough samples to run analysis. Skipping!")
        continue
    
    # Combine analysis-specific subject mask with site mask
    combined_mask = merge_bool_masks(subject_mask, site_mask)
    
    # Apply combined mask
    y_, groups_ = y[combined_mask], groups_vec[combined_mask]
    X_ = X[combined_mask, :]
    
    # Create CV splits
    cv_splits = repeated_group_stratified_KFold_splits(y_, groups_, 
                                                       n_folds=outer_cv_folds, n_repeats=outer_cv_repeats, 
                                                       seed=0, verbose=True)
    n_splits=len(cv_splits)

    # Initialize scoring evaluaters and classification pipeline
    scoring = Evaluater()
    metric_labels = scoring.evaluate_labels()
    metric_scores = np.zeros((n_splits, len(metric_labels)))

    best_params_lin = np.zeros((n_splits))
    
    y_predictions = np.ones((n_splits, len(y_))) * -1
    y_scores = np.ones((n_splits, len(y_))) * -1

    svm = SVC(kernel='precomputed', class_weight='balanced', C=1)
    svm_pipe = Pipeline([('centerer', KernelCenterer()), 
                         ('clf', svm)])

    # Precompute Kernels
    K_lin = pairwise_kernels(X_, metric='linear')
    
    t1 = time()    
    
    # Run cross-validation
    for i_split, (train_idx, test_idx) in enumerate(cv_splits):

        t3 = time()

        K_lin_train, K_lin_test = K_lin[np.ix_(train_idx, train_idx)], K_lin[np.ix_(test_idx, train_idx)]   
        y_train, y_test = y_[train_idx], y_[test_idx]
        groups_train = groups_[train_idx]

        nested_cv_splits = repeated_group_stratified_KFold_splits(y_train, groups_train, 
                                                                  n_folds=inner_cv_folds, n_repeats=inner_cv_repeats,
                                                                  seed=0, verbose=True)

        # Run gridsearch for linear Kernel (C)
        lin_model, lin_best_params, lin_best_score = run_gridsearch(K_lin_train, y_train, 
                                                                    svm_pipe, grid_params_lin_K, 
                                                                    nested_cv_splits, n_jobs=20, 
                                                                    scoring='balanced_accuracy', verbose=True)
        y_score = lin_model.decision_function(K_lin_test) 
        y_pred = lin_model.predict(K_lin_test)
        metric_scores[i_split, :] = scoring.evaluate_prediction(y_score=y_score, y_pred=y_pred, y_true=y_test)
        best_params_lin[i_split] = lin_best_params[0]
        y_predictions[i_split, test_idx] = y_pred
        y_scores[i_split, test_idx] = y_score

        t4 = time()

        print("Finished CV iteration {}/{} in {:.2f} minutes: ".format(i_split+1, n_splits, (t4-t3)/60.))

    t2 = time() 
    
    print()
    print("Finished classification in {:.2f} minutes".format((t2-t1)/60.))
    print()
    
    print("Performance averaged across splits:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores[:, m_idx].mean()))
    print()
        
    # Save classification results
    np.savez(os.path.join(save_dir, 'clf_results.npz'), 
             cv_splits=cv_splits,
             metric_scores=metric_scores,
             metric_labels=metric_labels,
             y_predictions=y_predictions,
             y_scores=y_scores,
             best_params_lin=best_params_lin,
             dtype=object
            )
    
    # Save results summary to DataFrame for quick overview
    results_df = pd.DataFrame([metric_scores.mean(axis=0)], columns=metric_labels)
    results_df.to_csv(os.path.join(save_dir, 'clf_summary.csv'))

In [ ]:
# 3b) Child/Early onset (coded=0) vs Late/Adult onset OCD (coded=1)
RESULTS_DIR = os.path.join(CLF_RESULTS_DIR, '3b_Age_of_Onset_Analyses')

N_jobs = 20 

title_template = '{}_CLF_{}_SAMPLES_{}_FEATURES'

# Only have to change these 3 variables for different subgroup analyses.
filter_mask = is_early_AO_pt_or_late_AO_pt_mask # Specific for analyses: only look into patients subgroups    
class_label = 'Age_of_Onset'
y = AO_vec.copy()
                           
# Now iterate over all age groups and feature sets
for age_group, feature_label in product(age_groups, feature_labels):
    
    title = title_template.format(class_label, age_group, feature_label)
    print(title)
    
    save_dir = os.path.join(RESULTS_DIR, title)
    
    if os.path.exists(save_dir):
        print("Output directory already exists, skipping...")
        continue
    
    ensure_folder(save_dir)
    
    X = feature_sets_dict[feature_label]
    
    # Merge subject masks
    age_group_mask = age_group_masks_dict[age_group]
    subject_mask = merge_bool_masks(age_group_mask, filter_mask)
    
    # Create copy of covariates df and append class_label vec is this is not included already
    tmp_covariates_df = included_subjects_covariates_df.copy()
    if class_label not in tmp_covariates_df.columns:
        tmp_covariates_df[class_label] = y
        
    # Filter out sites that have too little examples per class
    site_mask, counts_df = has_N_samples_per_class(tmp_covariates_df, class_label, subject_mask, 
                                                   N_threshold=10, verbose=True)
    
    # Save overview of included sites and class counts
    counts_df.to_csv(os.path.join(save_dir, 'class_counts.csv'), index=False)
    
    if counts_df.shape[0] < 2:
        print("Not enough samples to run analysis. Skipping!")
        continue
    
    # Combine analysis-specific subject mask with site mask
    combined_mask = merge_bool_masks(subject_mask, site_mask)
    
    # Apply combined mask
    y_, groups_ = y[combined_mask], groups_vec[combined_mask]
    X_ = X[combined_mask, :]
    
    # Create CV splits
    cv_splits = repeated_group_stratified_KFold_splits(y_, groups_, 
                                                       n_folds=outer_cv_folds, n_repeats=outer_cv_repeats, 
                                                       seed=0, verbose=True)
    n_splits=len(cv_splits)

    # Initialize scoring evaluaters and classification pipeline
    scoring = Evaluater()
    metric_labels = scoring.evaluate_labels()
    metric_scores = np.zeros((n_splits, len(metric_labels)))

    best_params_lin = np.zeros((n_splits))
    
    y_predictions = np.ones((n_splits, len(y_))) * -1
    y_scores = np.ones((n_splits, len(y_))) * -1

    svm = SVC(kernel='precomputed', class_weight='balanced', C=1)
    svm_pipe = Pipeline([('centerer', KernelCenterer()), 
                         ('clf', svm)])

    # Precompute Kernels
    K_lin = pairwise_kernels(X_, metric='linear')
    
    t1 = time()    
    
    # Run cross-validation
    for i_split, (train_idx, test_idx) in enumerate(cv_splits):

        t3 = time()

        K_lin_train, K_lin_test = K_lin[np.ix_(train_idx, train_idx)], K_lin[np.ix_(test_idx, train_idx)]   
        y_train, y_test = y_[train_idx], y_[test_idx]
        groups_train = groups_[train_idx]

        nested_cv_splits = repeated_group_stratified_KFold_splits(y_train, groups_train, 
                                                                  n_folds=inner_cv_folds, n_repeats=inner_cv_repeats,
                                                                  seed=0, verbose=True)

        # Run gridsearch for linear Kernel (C)
        lin_model, lin_best_params, lin_best_score = run_gridsearch(K_lin_train, y_train, 
                                                                    svm_pipe, grid_params_lin_K, 
                                                                    nested_cv_splits, n_jobs=20, 
                                                                    scoring='balanced_accuracy', verbose=True)
        y_score = lin_model.decision_function(K_lin_test) 
        y_pred = lin_model.predict(K_lin_test)
        metric_scores[i_split, :] = scoring.evaluate_prediction(y_score=y_score, y_pred=y_pred, y_true=y_test)
        best_params_lin[i_split] = lin_best_params[0]
        y_predictions[i_split, test_idx] = y_pred
        y_scores[i_split, test_idx] = y_score

        t4 = time()

        print("Finished CV iteration {}/{} in {:.2f} minutes: ".format(i_split+1, n_splits, (t4-t3)/60.))

    t2 = time() 
    
    print()
    print("Finished classification in {:.2f} minutes".format((t2-t1)/60.))
    print()
    
    print("Performance averaged across splits:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores[:, m_idx].mean()))
    print()
        
    # Save classification results
    np.savez(os.path.join(save_dir, 'clf_results.npz'), 
             cv_splits=cv_splits,
             metric_scores=metric_scores,
             metric_labels=metric_labels,
             y_predictions=y_predictions,
             y_scores=y_scores,
             best_params_lin=best_params_lin,
             dtype=object
            )
    
    # Save results summary to DataFrame for quick overview
    results_df = pd.DataFrame([metric_scores.mean(axis=0)], columns=metric_labels)
    results_df.to_csv(os.path.join(save_dir, 'clf_summary.csv'))

In [ ]:
# 3c) High Severity (coded=1) vs Low Severity OCD (coded=0)
RESULTS_DIR = os.path.join(CLF_RESULTS_DIR, '3c_Severity_Analyses')

N_jobs = 20 

title_template = '{}_CLF_{}_SAMPLES_{}_FEATURES'

# Only have to change these 3 variables for different subgroup analyses.
filter_mask = is_low_sev_pt_or_high_sev_pt_mask # Specific for analyses: only look into patients subgroups.      
class_label = 'Severity'
y = sev_vec.copy()
                           
# Now iterate over all age groups and feature sets
for age_group, feature_label in product(age_groups, feature_labels):
    
    title = title_template.format(class_label, age_group, feature_label)
    print(title)
    
    save_dir = os.path.join(RESULTS_DIR, title)
    
    if os.path.exists(save_dir):
        print("Output directory already exists, skipping...")
        continue
    
    ensure_folder(save_dir)
    
    X = feature_sets_dict[feature_label]
    
    # Merge subject masks
    age_group_mask = age_group_masks_dict[age_group]
    subject_mask = merge_bool_masks(age_group_mask, filter_mask)
    
    # Create copy of covariates df and append class_label vec is this is not included already
    tmp_covariates_df = included_subjects_covariates_df.copy()
    if class_label not in tmp_covariates_df.columns:
        tmp_covariates_df[class_label] = y
        
    # Filter out sites that have too little examples per class
    site_mask, counts_df = has_N_samples_per_class(tmp_covariates_df, class_label, subject_mask, 
                                                   N_threshold=10, verbose=True)
    
    # Save overview of included sites and class counts
    counts_df.to_csv(os.path.join(save_dir, 'class_counts.csv'), index=False)
    
    if counts_df.shape[0] < 2:
        print("Not enough samples to run analysis. Skipping!")
        continue
    
    # Combine analysis-specific subject mask with site mask
    combined_mask = merge_bool_masks(subject_mask, site_mask)
    
    # Apply combined mask
    y_, groups_ = y[combined_mask], groups_vec[combined_mask]
    X_ = X[combined_mask, :]
    
    # Create CV splits
    cv_splits = repeated_group_stratified_KFold_splits(y_, groups_, 
                                                       n_folds=outer_cv_folds, n_repeats=outer_cv_repeats, 
                                                       seed=0, verbose=True)
    n_splits=len(cv_splits)

    # Initialize scoring evaluaters and classification pipeline
    scoring = Evaluater()
    metric_labels = scoring.evaluate_labels()
    metric_scores = np.zeros((n_splits, len(metric_labels)))

    best_params_lin = np.zeros((n_splits))
    
    y_predictions = np.ones((n_splits, len(y_))) * -1
    y_scores = np.ones((n_splits, len(y_))) * -1

    svm = SVC(kernel='precomputed', class_weight='balanced', C=1)
    svm_pipe = Pipeline([('centerer', KernelCenterer()), 
                         ('clf', svm)])

    # Precompute Kernels
    K_lin = pairwise_kernels(X_, metric='linear')
    
    t1 = time()    
    
    # Run cross-validation
    for i_split, (train_idx, test_idx) in enumerate(cv_splits):

        t3 = time()

        K_lin_train, K_lin_test = K_lin[np.ix_(train_idx, train_idx)], K_lin[np.ix_(test_idx, train_idx)]   
        y_train, y_test = y_[train_idx], y_[test_idx]
        groups_train = groups_[train_idx]

        nested_cv_splits = repeated_group_stratified_KFold_splits(y_train, groups_train, 
                                                                  n_folds=inner_cv_folds, n_repeats=inner_cv_repeats,
                                                                  seed=0, verbose=True)

        # Run gridsearch for linear Kernel (C)
        lin_model, lin_best_params, lin_best_score = run_gridsearch(K_lin_train, y_train, 
                                                                    svm_pipe, grid_params_lin_K, 
                                                                    nested_cv_splits, n_jobs=20, 
                                                                    scoring='balanced_accuracy', verbose=True)
        y_score = lin_model.decision_function(K_lin_test) 
        y_pred = lin_model.predict(K_lin_test)
        metric_scores[i_split, :] = scoring.evaluate_prediction(y_score=y_score, y_pred=y_pred, y_true=y_test)
        best_params_lin[i_split] = lin_best_params[0]
        y_predictions[i_split, test_idx] = y_pred
        y_scores[i_split, test_idx] = y_score

        t4 = time()

        print("Finished CV iteration {}/{} in {:.2f} minutes: ".format(i_split+1, n_splits, (t4-t3)/60.))

    t2 = time() 
    
    print()
    print("Finished classification in {:.2f} minutes".format((t2-t1)/60.))
    print()
    
    print("Performance averaged across splits:")
    for m_idx, m_label in enumerate(metric_labels):
        print("mean {}: {:.2f}".format(m_label, metric_scores[:, m_idx].mean()))
    print()
        
    # Save classification results
    np.savez(os.path.join(save_dir, 'clf_results.npz'), 
             cv_splits=cv_splits,
             metric_scores=metric_scores,
             metric_labels=metric_labels,
             y_predictions=y_predictions,
             y_scores=y_scores,
             best_params_lin=best_params_lin,
             dtype=object
            )
    
    # Save results summary to DataFrame for quick overview
    results_df = pd.DataFrame([metric_scores.mean(axis=0)], columns=metric_labels)
    results_df.to_csv(os.path.join(save_dir, 'clf_summary.csv'))

In [ ]:
CLF_RESULTS_DIR

In [ ]:
# Parse clasification performances without permutations

In [ ]:
result_paths = np.array(sorted(glob(os.path.join(CLF_RESULTS_DIR, '*', '*', 'clf_results.npz'))))

In [ ]:
clf_metrics_to_report = ['AUC', 'balanced_accuracy', 'sensitivity', 'specificity', 
                         'positive_predictive_value', 'negative_predictive_value']

pooled_results_columns = np.append(['analysis_label', 'filter_str', 'age_group', 'feature_set'], 
                                   clf_metrics_to_report)
pooled_results_df = pd.DataFrame(columns=pooled_results_columns)

for result_path in result_paths:
    
    result_dir = os.path.dirname(result_path)
    
    analysis_label = result_dir.split('/')[-2]
    clf_title = os.path.basename(result_dir)
    
    clf_task = clf_title.split('_CLF_')[0]
    age_group = clf_title.split('_CLF_')[1].split('_SAMPLES_')[0]
    feature_set = clf_title.split('_CLF_')[1].split('_SAMPLES_')[1].split('_FEATURES')[0]

    filter_string = clf_title.split('_FEATURES_')[1].split('_FILTER')[0] if '_FILTER' in clf_title else ''
    
    # Load neutral permutation (actual obtained classification result)
    clf_result = os.path.join(result_dir, 'clf_results.npz')
    clf_metrics = np.load(clf_result, allow_pickle=True)['metric_scores']
    metric_labels = np.load(clf_result, allow_pickle=True)['metric_labels']
    
    # Sanity check here!
    results_df = pd.read_csv(os.path.join(result_dir, 'clf_summary.csv'))
    
    assert np.isclose(results_df.AUC.values[0], clf_metrics[:, metric_labels=='AUC'].mean())
    
    # Extract N used for classification to compute confidence intervals
    N = np.load(clf_result, allow_pickle=True)['y_predictions'].shape[-1]
    
    clf_metrics_formatted = [np.round(clf_metrics[:, metric_labels==l].mean(), 3) for l in clf_metrics_to_report]

    tmp_df = pd.DataFrame(columns=pooled_results_columns,
                          data=np.c_[[analysis_label], [filter_string], [age_group], [feature_set],  
                                     [clf_metrics_formatted]])

    pooled_results_df = pd.concat([pooled_results_df, tmp_df], ignore_index=True)

In [ ]:
filename = 'clf_results_wo_perms_' + datetime.date.today().strftime("%d_%m_%Y") + '.csv'
pooled_results_df.to_csv(os.path.join(CLF_RESULTS_DIR, filename))